# Домашнее задание к 21.09 - Бобкова Ксения

В программе используется файл с уже собранными отзывами про компьютеры и телефоны

In [5]:
import json
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from itertools import chain
from collections import Counter

morph = MorphAnalyzer()

final_texts_positive = []
final_texts_negative = []

#Работа с текстами уже размеченных отзывово из файла (токенизация, лемматизация,изменение регистра) 
#И разбиваем на 2 категории 
with open('reviews.json', 'r', encoding="utf-8") as f: 
    data = json.load(f)
    for r in data["reviews"]:
        final = []
        text = r['text']
        words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
        for word in words:
            i = []
            i = morph.parse(word)[0].normal_form
            final.append(i)
            i = []
        if r["class"] == "positive":
            final_texts_positive.append(final)
        else:
            final_texts_negative.append(final)

#из списка списков делаем один список
list_positive = list(chain.from_iterable(final_texts_positive))
list_negative = list(chain.from_iterable(final_texts_negative))

#подчситываем сколько раз каждое слово встретилось в той или иной категории отзывов
count_positive = Counter(list_positive)
count_negative = Counter(list_negative)

#Фильтруем получившиеся словари (убираем все слова, которые встретились 1 раз)
for w,num in list(count_negative.items()):
    if num == 1:
        del count_negative[w]

for ww,numm in list(count_positive.items()):
    if numm == 1:
        del count_positive[ww]

#Создаем 2 списка с уникальными словами для каждой категории
positive = []
negative = []

for key,value in count_positive.items():
    if key not in count_negative:
        positive.append(key)

for key,value in count_negative.items():
    if key not in positive:
        negative.append(key)


#Функция, которая на вход получает текст отзыва, а возвращает его категорию
def marker(rev_t):
    token_text = []
    words = [w.lower() for w in word_tokenize(rev_t) if w.isalpha()]
    for word in words:
        i = []
        i = morph.parse(word)[0].normal_form
        token_text.append(i)
        i = []
    counter_p = 0
    counter_n = 0

    for token in token_text:
        if token in positive:
            counter_p += 1
        elif token in negative:
            counter_n += 1
    if counter_p > counter_n:
        mark = "positive"
    else:
        mark = "negative"

    return  mark


#проверка не размеченных ранее отзывов
with open('checher.json', 'r', encoding="utf-8") as c:
    checker = json.load(c)
    for rev in data["reviews"]:
        text_check = rev['text']
        print(marker(text_check))
